## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Описание проекта

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. 
С этим датасетом вы уже работали в четвёртой теме курса. 
В колонках датасета содержатся следующие данные:
    
longitude — широта;

latitude — долгота;

housing_median_age — медианный возраст жителей жилого массива;

total_rooms — общее количество комнат в домах жилого массива;

total_bedrooms — общее количество спален в домах жилого массива;

population — количество человек, которые проживают в жилом массиве;

households — количество домовладений в жилом массиве;

median_income — медианный доход жителей жилого массива;

median_house_value — медианная стоимость дома в жилом массиве;

ocean_proximity — близость к океану.

На основе данных нужно предсказать медианную стоимость дома в жилом массиве — median_house_value. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml import Pipeline


from pyspark.ml.feature import  StringIndexer, OneHotEncoder, StandardScaler,VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
RND= 1234

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

# Инициализируем локальную Spark-сессию

In [2]:
spark = SparkSession.builder \
                    .master("local")\
                    .appName("CA House") \
                    .getOrCreate()

# Подготовка данных

In [3]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 
df.printSchema() 
df.show(5)


root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR B

In [4]:
df.describe().toPandas().set_index('summary')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
summary,,,,,,,,,,
count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Из представленных данных, все столбцы являются числовыми, кроме столбца ocean_proximity, который имеет формат string    (категориональные значения).<span style="color:red"> столбцы longitude and latitude являются кординатами и не будут влиять на нашу задачу, удалим их.</span>
Также, из представленных данных видно, что в столбце housing_median_age и median_income очень небольшие значения

Проверим пропуски в данных

In [5]:
for column in df.columns:
        is_null = F.isnull(F.col(column)) | F.isnan(F.col(column))
        print(column, df.filter(is_null).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Кол-во пропусков в столбце total_bedrooms=207 

 заменим пропуски медианным значением

In [6]:
bedrooms_median = df.select('total_bedrooms').toPandas().median()[0]

bedrooms_median 

435.0

In [7]:
df = df.na.fill(bedrooms_median, subset=["total_bedrooms"])

In [8]:
for column in df.columns:
        is_null = F.isnull(F.col(column)) | F.isnan(F.col(column))
        print(column, df.filter(is_null).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


In [9]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Заменили пропуски медианным значением. среднее значение изменилось не существено


Разделим данные на числовые и категориаональные признаки

In [10]:
cat_cols = ['ocean_proximity']
num_cols  = df.drop('ocean_proximity', 'median_house_value').columns

target = "median_house_value"

Разделим данные на тренировочную и тестовую выборку

In [11]:
df_train, df_test = df.randomSplit([.8,.2], seed=RND)
print(df_train.count(), df_test.count())

16525 4115


Преобразуем катеориональные признаки в числовые, используя метод StringIndexer.
Далее применим метод OHE
объеденим признаки методом VectorAssembler
и соберем это все в один pipeline

In [12]:
indexer= StringIndexer(inputCols=cat_cols, outputCols=[col + '_idx' for col in cat_cols])

In [13]:
encoder = OneHotEncoder(inputCols=[col + '_idx' for col in cat_cols], outputCols=[col + '_ohe' for col in cat_cols])

In [14]:
numerical_assembler = VectorAssembler(inputCols=num_cols, outputCol='numerical_features')

In [15]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")

In [16]:
final_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe', 'numerical_features_scaled'], 
                                  outputCol="features")

In [17]:
pipeline = Pipeline(stages=[indexer, encoder, numerical_assembler,standardScaler,final_assembler])

# Обучение моделей

Построим две модели линейной регрессии: все данные и только числовые 

In [18]:
lr_1 = LinearRegression(labelCol=target, featuresCol='features',regParam=0.0)

pipeline_1 = Pipeline(stages=[indexer, encoder, numerical_assembler,standardScaler,final_assembler,lr_1])





In [19]:
predictions_1 = pipeline_1.fit(df_train).transform(df_test)

24/02/15 13:51:08 WARN Instrumentation: [32aa503f] regParam is zero, which might cause numerical instability and overfitting.
24/02/15 13:51:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/02/15 13:51:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/02/15 13:51:09 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/02/15 13:51:09 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [22]:
predictions_1.toPandas().head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx,ocean_proximity_ohe,numerical_features,numerical_features_scaled,features,prediction
0,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0,NEAR OCEAN,2.0,"(0.0, 0.0, 1.0, 0.0)","[-124.3, 41.8, 19.0, 2672.0, 552.0, 1298.0, 47...","[-62.04383622990429, 19.61128612892717, 1.5073...","[0.0, 0.0, 1.0, 0.0, -62.04383622990429, 19.61...",116671.739892
1,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0,NEAR OCEAN,2.0,"(0.0, 0.0, 1.0, 0.0)","[-124.26, 40.58, 52.0, 2217.0, 394.0, 907.0, 3...","[-62.02387039362757, 19.03889930889628, 4.1253...","[0.0, 0.0, 1.0, 0.0, -62.02387039362757, 19.03...",191472.506751
2,-124.18,40.79,39.0,1836.0,352.0,883.0,337.0,1.7450,70500.0,NEAR OCEAN,2.0,"(0.0, 0.0, 1.0, 0.0)","[-124.18, 40.79, 39.0, 1836.0, 352.0, 883.0, 3...","[-61.98393872107413, 19.137424909065533, 3.094...","[0.0, 0.0, 1.0, 0.0, -61.98393872107413, 19.13...",144580.640718
3,-124.17,40.74,17.0,2026.0,338.0,873.0,313.0,4.0357,128900.0,NEAR OCEAN,2.0,"(0.0, 0.0, 1.0, 0.0)","[-124.17, 40.74, 17.0, 2026.0, 338.0, 873.0, 3...","[-61.97894726200495, 19.11396643283476, 1.3486...","[0.0, 0.0, 1.0, 0.0, -61.97894726200495, 19.11...",208330.742251
4,-124.17,40.75,13.0,2171.0,339.0,951.0,353.0,4.8516,116100.0,NEAR OCEAN,2.0,"(0.0, 0.0, 1.0, 0.0)","[-124.17, 40.75, 13.0, 2171.0, 339.0, 951.0, 3...","[-61.97894726200495, 19.118658128080913, 1.031...","[0.0, 0.0, 1.0, 0.0, -61.97894726200495, 19.11...",235023.167524


In [20]:
lr_2 = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled',regParam=0.0)

pipeline_2 = Pipeline(stages=[numerical_assembler,standardScaler,lr_2])



In [21]:
predictions_2 = pipeline_2.fit(df_train).transform(df_test)

24/02/15 13:54:03 WARN Instrumentation: [1d93d5d7] regParam is zero, which might cause numerical instability and overfitting.


In [23]:
predictions_2.toPandas().head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,numerical_features,numerical_features_scaled,prediction
0,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0,NEAR OCEAN,"[-124.3, 41.8, 19.0, 2672.0, 552.0, 1298.0, 47...","[-62.04383622990429, 19.61128612892717, 1.5073...",65465.098032
1,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0,NEAR OCEAN,"[-124.26, 40.58, 52.0, 2217.0, 394.0, 907.0, 3...","[-62.02387039362757, 19.03889930889628, 4.1253...",164077.395803
2,-124.18,40.79,39.0,1836.0,352.0,883.0,337.0,1.7450,70500.0,NEAR OCEAN,"[-124.18, 40.79, 39.0, 1836.0, 352.0, 883.0, 3...","[-61.98393872107413, 19.137424909065533, 3.094...",110573.835686
3,-124.17,40.74,17.0,2026.0,338.0,873.0,313.0,4.0357,128900.0,NEAR OCEAN,"[-124.17, 40.74, 17.0, 2026.0, 338.0, 873.0, 3...","[-61.97894726200495, 19.11396643283476, 1.3486...",174719.352247
4,-124.17,40.75,13.0,2171.0,339.0,951.0,353.0,4.8516,116100.0,NEAR OCEAN,"[-124.17, 40.75, 13.0, 2171.0, 339.0, 951.0, 3...","[-61.97894726200495, 19.118658128080913, 1.031...",201406.288895


# Анализ результатов

 определяем метрики RMSE, MAE, R2 для модели 1 (все данные)
 

In [24]:
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_1)

evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_1)

evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_1)

print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)



RMSE: 67359.91058072698
MAE: 49218.92567022398
R2: 0.6542930056011936


 определяем метрики RMSE, MAE, R2 для модели 2 (количественные)
    

In [25]:
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_2)

evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_2)

evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_2)

print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

RMSE: 68177.39948274654
MAE: 50396.989759948454
R2: 0.6458509950600302


На основании полученных данных видно, что первая модель выглядит лучше по всем  3 параметрам :
RMSE  и MAE (чем ближе к нулю, тем более точнее предсказание ), хотя и в первом и во втором случае показатели достаточно большие
R2,чем ближе к единице, тем модель точнее. в первом случае показатель составил 0,6543

# Итог

1) Инициализировали локальную Spark-сессию.

2) прочитали и подготовили данные из /datasets/housing.csv. Нашили пропуску в столбце total_bedrooms, кол-во составило 207. заменили данные пропуски на медианное значение (которое составило 435 )

3) преобразовали колонки с категориональными и количественными данными методами StringIndexer,OneHotEncoder,StandardScaler и пр

4) обучили две модели : со всеми данными и количественными

5) определили метрики по двум моделям и сделали вывод, что по всем трем показателям модель, которая включает в себя все данные, показала наилучший результат